In [1]:
''' VMP 2022-03-05: 
based on Analysis_RASMUS.ipynb
some changes to make it work & some warnings.
double-check with Nielbo. 
'''

' VMP 2022-03-05: \nbased on Analysis_RASMUS.ipynb\nsome changes to make it work & some warnings.\ndouble-check with Nielbo. \n'

In [21]:
#### Imports
import pandas as pd 
import pickle as pkl 
import altair as alt
import numpy as np 
import os 
import re
import pyLDAvis
import gensim
import openpyxl

/home/coder/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Load data

In [3]:
# function which is basically the same as in "helper_functions" but manages new paths 
def prep_data(filepath):
    """Preparing the data

    Returns:
        pd.Dataframes: Returns three dataframes - the whole dataset, the diplomats and the media.
    """    
    en_df = pd.read_csv(filepath)

    en_df['month'] = pd.DatetimeIndex(en_df['created_at']).month
    en_df['date'] = pd.DatetimeIndex(en_df['created_at']).date
    
    
    def retweet_binary(string):
        if string == "retweeted":
            return "Retweet"
        else:
            return "Other"

    en_df["retweet_bin"] = en_df["retweet"].apply(lambda x: retweet_binary(x))
    
    en_df["date"] = pd.to_datetime(en_df["date"])
    
    return en_df, en_df[en_df["category"] == "Diplomat"].reset_index(drop=True), en_df[en_df["category"] == "Media"].reset_index(drop=True)
    

/home/coder/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
data, diplomats, media = prep_data('../data/all_data.csv')

/home/coder/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Load models

In [6]:
def load_models(filepath_media, filepath_diplomat):
    with open(filepath_media, "rb") as f:
        media = pkl.load(f)
    
    with open(filepath_diplomat, "rb") as f:
        diplo = pkl.load(f)
        
    return media, diplo

/home/coder/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
media_dict, diplo_dict = load_models('../data/models/alldataMedia.pkl', '../data/models/alldataDiplomat.pkl')

/home/coder/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
# diplomats without retweets
diplo_without = diplomats[diplomats["retweet"] != "retweeted"].reset_index(drop = True)

/home/coder/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# VMP 2022-03-05
# for the below I have just used the code in Analysis_RASMUS.ipynb
# not sure whether diplo_dict() should be filtered for retweets?
# made some changes

In [20]:
# Task 1: Diplomat document representation and metadata
## add $\theta$ to to diplomats

## comment0: diplo_without, model is trained without retweet to avoid redundancy
## comment1: corpus is filtered for extremes with Gensim 
## comment2 (VMP 2022-03-05): using max_topics, rather than len(theta[0]). 

# predict theta_i on document in corpus 
print(f'\n[INFO] predicting \u03B8...\n{"*"*25}')
verbose = 5000
theta = list()
for i, doc in enumerate(diplo_dict['corpus']):
    vector = diplo_dict['model'][doc]
    theta.append(vector[0])
    if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
        print(f'[INFO] processed {i + 1}/{len(diplo_dict["corpus"])}')
# dimension consistent theta array
ntopics_lst = [len(doc) for doc in theta]
max_topics = max(ntopics_lst)
print(f"maximum number of topics: {max_topics}")
(m, k) = (len(theta), max_topics) #kappa VMP2022-03-05: (previously: (len(theta), len(theta[0])))
theta_transform = np.zeros((m, k))
for (i, doc) in enumerate(theta):
    theta_i = np.zeros((1,30)) # VMP: where is this used?
    for j, var in doc:
        theta_transform[i,j] = var
# write to csv
df = pd.DataFrame()
for i in range(len(theta_transform[0,:])):
    df[f'var {i}'] = theta_transform[:,i]
dfout = pd.concat([diplo_without, df], axis=1)
fname = os.path.join('..', 'data', 'theta','diplomats_noretweet_theta_df_all.csv')
dfout.to_csv(fname, index=False)
print(f'{"*"*25}\n[INFO] data exported to {fname}')
fname_maxqda = '..' + fname.split('.')[2] + '.xlsx' # VMP2022-03-05: (previously: fname.split('.')[0] + '.xlsx')
dfout['created_at'] = dfout['created_at'].astype(str)
dfout.to_excel(fname_maxqda, index=False)
print(f'[INFO] data exported to {fname_maxqda}')

/home/coder/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

[INFO] predicting θ...
*************************
[INFO] processed 5000/67026
[INFO] processed 10000/67026
[INFO] processed 15000/67026
[INFO] processed 20000/67026
[INFO] processed 25000/67026
[INFO] processed 30000/67026
[INFO] processed 35000/67026
[INFO] processed 40000/67026
[INFO] processed 45000/67026
[INFO] processed 50000/67026
[INFO] processed 55000/67026
[INFO] processed 60000/67026
[INFO] processed 65000/67026
maximum number of topics: 30
*************************
[INFO] data exported to ../data/theta/diplomats_noretweet_theta_df_all.csv
[INFO] data exported to ../data/theta/diplomats_noretweet_theta_df_all.xlsx
